In [12]:
import pandas as pd
from collections import Counter
import random

In [13]:
df = pd.read_csv('tcc_ceds_music.csv')

In [14]:
df = df[['artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len', 'topic']]
myPlayList = []

In [15]:
def filter_records(df):
    # Display genre choices
    print("Choose a genre:")
    genres = df['genre'].unique()
    for i, genre in enumerate(genres):
        print(f"{i+1}. {genre}")
    
    # Get user genre choice
    genre_choice = input("Enter the number corresponding to your choice: ")
    if not genre_choice.isdigit() or int(genre_choice) < 1 or int(genre_choice) > len(genres):
        print("Invalid choice.")
        return None
    selected_genre = genres[int(genre_choice) - 1]
    print(f"Selected genre: {selected_genre}")

    # Display topic choices
    print("\nChoose a topic:")
    topics = df['topic'].unique()
    for i, topic in enumerate(topics):
        print(f"{i+1}. {topic}")
    
    # Get user topic choice
    topic_choice = input("Enter the number corresponding to your choice: ")
    if not topic_choice.isdigit() or int(topic_choice) < 1 or int(topic_choice) > len(topics):
        print("Invalid choice.")
        return None
    selected_topic = topics[int(topic_choice) - 1]
    print(f"Selected topic: {selected_topic}")

    # Display release date range choices
    print("\nChoose a release date range:")
    print("1. 1950-1959")
    print("2. 1960-1969")
    print("3. 1970-1979")
    print("4. 1980-1989")
    print("5. 1990-1999")
    print("6. 2000-2009")
    print("7. 2010-2019")
    # Add more ranges as needed
    
    # Get user release date range choice
    release_date_choice = input("Enter the number corresponding to your choice: ")
    if not release_date_choice.isdigit() or int(release_date_choice) < 1 or int(release_date_choice) > 7:
        print("Invalid choice.")
        return None
    start_year = int(release_date_choice) * 10 + 1940
    end_year = start_year + 9
    print(f"Selected release date range: {start_year}-{end_year}")

    # Filter the DataFrame based on the user's choices
    filtered_df = df[(df['genre'] == selected_genre) & (df['topic'] == selected_topic) & (df['release_date'].between(start_year, end_year)) & (df['len'] >= 100)]
    
    print(f"\nNumber of matching records: {len(filtered_df)}")

    return filtered_df

    
    
    

In [16]:
def show(musicList):
    if musicList is None or len(musicList) == 0:
        print("You don't have any song in your playlist, try adding new songs")
        return 
    
    for i, song in enumerate(musicList):
        print(i + 1, "Song title:", song['track_name'])
        print("Artist name:", song['artist_name'])
        print("Genre:", song['genre'])
        print("Release year:", song['release_date'], '\n')

In [17]:
def addSong():
    #a lambda function to check if there is any duplicates in the playlist 
    is_dup = lambda song: any(s['lyrics'] == song['lyrics'] for s in myPlayList)


    #filter the dataframe to get recomandations based on his choices
    recomends = filter_records(df)

    if recomends is None or len(recomends) == 0:
        print("sorry we couldn't find anything that fits your taste :'(")
        return
    
    recomends = recomends.to_dict(orient='records')
    
    print("\nHere is what we think you would like")
    show(recomends)

    repeat = 'yes'
    while repeat.lower() == 'yes' or repeat.lower() == 'y':
    #User chooses one number of the available songs to add it
        while True:
            try:
                songID = int(input("Please choose the song number"))
                
                if songID > 0 and songID and songID <= len(recomends):
                    #check if the song is already in myPlayList
                    if not is_dup(recomends[songID - 1]):
                        break
                    else:
                        songID = -1
                        print("You already have this song in your Playlist")
                        break
                else:
                    print("Please enter a number from the above songs")

            except ValueError:
                print("please enter a number")

        #add the selected song by index into the user's playList
        if songID != -1:
            myPlayList.append(recomends[songID - 1])
            #show the newly added song for the user
            print("\nYou added the following song successfully:")
            show([recomends[songID-1]])

        repeat = input("Do you want to keep adding, (yes/no)?")



In [18]:
def printLyrics():
    show(myPlayList)
    if myPlayList is None or len(myPlayList) == 0:
        return
    
    while True:
        try:
            songID = int(input("Please choose the song number"))
            
            if songID > 0 and songID <= len(myPlayList):
                song = myPlayList[songID - 1]
                lyrics = song['lyrics']

                words = lyrics.split()
                #print the lyrics where each line contain only 8 words
                for i, word in enumerate(words):
                    print(word, end=' ')
                    if (i + 1) % 8 == 0:
                        print()
                print()
                break
            else:
                print("Please enter a number from the above songs")

        except ValueError:
            print("please enter a number")

In [19]:
def randomPlayList():
    while True:
        try:
            size = int(input("Please enter how many songs do you want in your playlist: ")) 
            if size > 0:
                break
            else:
                print("please enter a number larger than 0")
        except:
            print("please enter a number")

    recomands = filter_records(df)

    if recomands is None or len(recomands) == 0:
        print("sorry we couldn't find anything that fits your taste :'(")
        return
    

    if size > len(recomands):
        print("Sorry we only found", len(recomands), "songs, we added them to your list")

        myPlayList.extend(recomands.to_dict(orient='records'))
        
        print("\nHere is your new Playlist enjoy :)")
        show(myPlayList)
        return
    
    recomands = recomands.sample(size)
    myPlayList.extend(recomands.to_dict(orient='records'))

    print("\nHere is your new Playlist enjoy :)")
    show(myPlayList)
        
        




In [20]:
def delete(myPlayList) :
    if len(myPlayList) == 0:
        print("Your playlist is empty...")
        return
    
    print("What do you want to sing")
    show(myPlayList)
    while True:
        try:
            index_to_delete = int(input("Which song do you want to delete?"))

            if index_to_delete > 0 and index_to_delete <= len(myPlayList):
                deleted_song = myPlayList.pop(index_to_delete - 1)

                print("The following song deleted successfully:")
                show([deleted_song])

                print("Here is your Playlist after deleting:")
                show(myPlayList)

                return
            else:
                print("Please enter a number from the above songs")

        except:
            print("please enter a number")
            

In [29]:
def add_random_music(playList) :
    # Extract all genres
    if len(playList) == 0:
        
        recomends = df.sample(5).reset_index(drop = True)
        recomends = recomends.to_dict(orient='records')
    
        print("\nHere is what we think you would like")
        show(recomends)
        while True:
            try:
                songID = int(input("Please choose the song number"))

                if songID > 0 and songID <= len(recomends):
                    break
                else:
                    print("Please enter a number from the above songs")

            except ValueError:
                print("please enter a number")


        #add the selected song by index into the user's playList
        myPlayList.append(recomends[songID-1])
        #show the newly added song for the user
        print("\nYou added the following song successfully:")        
        
        show([recomends[songID-1]])
        del recomends[songID - 1]
        
        while True:
            if len(recomends) == 0:
                print('List is empty')
                break
            else:############
                choices = input("Do you want to keep adding, (yes/no)?")
                if choices.lower() == 'no' or choices.lower == 'n' :
                    break
                elif choices.lower() == 'yes' or choices.lower() == 'y':
                    show(recomends)
                    while True:
                        try:
                            songID = int(input("Please choose the song number"))

                            if songID > 0 and songID and songID <= len(recomends):
                                break
                            else:
                                print("Please enter a number from the above songs")

                        except ValueError:
                            print("please enter a number")


                    #add the selected song by index into the user's playList
                    myPlayList.append(recomends[songID-1])
                    #show the newly added song for the user
                    print("\nYou added the following song successfully:")


                    show([recomends[songID-1]])
                    del recomends[songID - 1]



    # Extract all genres   
    else:
        
        genres = [track['genre'] for track in playList]

        # Count the occurrences of each genre
        genre_counts = Counter(genres)

        # Return the max count 
        max_count = max(genre_counts.values())

        # Find the genre with the maximum count
        max_genres = [genre for genre, count in genre_counts.items() if count == max_count]

        # Randomly choose one of the genres with the maximum count
        chosen_genre = random.choice(max_genres)
        filteredDf = df[(df['genre'] == chosen_genre)]        
        recomends = filteredDf.sample(5).reset_index(drop = True)
        recomends = recomends.to_dict(orient='records')
    
        print("\nHere is what we think you would like")
        show(recomends)
        while True:
            try:
                songID = int(input("Please choose the song number"))

                if songID > 0 and songID and songID <= len(recomends):
                    break
                else:
                    print("Please enter a number from the above songs")

            except ValueError:
                print("please enter a number")


        #add the selected song by index into the user's playList
        myPlayList.append(recomends[songID-1])
        #show the newly added song for the user
        print("\nYou added the following song successfully:")
        
        
        show([recomends[songID-1]])
        del recomends[songID - 1]
        
        
        while True:
            if len(recomends) == 0:
                print('List is empty')
                break
            else:############
                choices = input("Do you want to keep adding, (yes/no)?")
                if choices.lower() == 'no' or choices.lower == 'n' :
                    break
                elif choices.lower() == 'yes' or choices.lower() == 'y':
                    show(recomends)
                    while True:
                        try:
                            songID = int(input("Please choose the song number"))

                            if songID > 0 and songID and songID <= len(recomends):
                                break
                            else:
                                print("Please enter a number from the above songs")

                        except ValueError:
                            print("please enter a number")


                    #add the selected song by index into the user's playList
                    myPlayList.append(recomends[songID-1])
                    #show the newly added song for the user
                    print("\nYou added the following song successfully:")


                    show([recomends[songID-1]])
                    del recomends[songID - 1]


In [30]:
print("\n***************Welcome to MUSICY***************")
print("            Where you discover music              ")
while True:
    print("\nOptions:")
    print("1. Add a song")
    print("2. Feeling lucky?")
    print("3. Expand your playlist")
    print("4. Delete a song from the playlist")
    print("5. Show playlist")
    print("6. Sing along")
    print("7. Exit \n")

    choice = input("Enter your choice: ")

    if choice == '1':
        addSong()
    elif choice == '2':
        randomPlayList()
    elif choice == '3':
        add_random_music(myPlayList)
        print("Here is your new playlist:")
        show(myPlayList)
    elif choice == '4':
        delete(myPlayList)
    elif choice == '5':
        show(myPlayList)
    elif choice == '6':
        printLyrics()
    elif choice == '7':
        print("Exiting...")
        break
    else:
        print("Invalid choice. Please enter a number between 1 and 7.")


***************Welcome to MUSICY***************
            Where you discover music              

Options:
1. Add a song
2. Feeling lucky?
3. Expand your playlist
4. Delete a song from the playlist
5. Show playlist
6. Sing along
7. Exit 


Here is what we think you would like
1 Song title: welcome to the club
Artist name: tim mcgraw
Genre: country
Release year: 1993 

2 Song title: reckoning
Artist name: whiskey myers
Genre: country
Release year: 2014 

3 Song title: million miles
Artist name: adam hood
Genre: country
Release year: 2004 

4 Song title: to think you've chosen me
Artist name: marty robbins
Genre: country
Release year: 1961 

5 Song title: long day comin'
Artist name: hal ketchum
Genre: country
Release year: 1991 


You added the following song successfully:
1 Song title: welcome to the club
Artist name: tim mcgraw
Genre: country
Release year: 1993 

1 Song title: reckoning
Artist name: whiskey myers
Genre: country
Release year: 2014 

2 Song title: million miles
Artis

In [ ]:
# recomend song function do you want to continue from 1 or 2 to yes and no
# show 1.